# Đề bài về nhà

## Yêu cầu

- Tự viết code cho mô hình Linear Regression theo công thức đã được dạy trong buổi lý thuyết trên lớp.
- Tự viết hàm dự đoán.
- Huấn luyện cả mô hình của thư viện và mô hình mình tự viết.
- In ra các trọng số: w0, w1, w2, ..., wn của cả 2 mô hình đã huấn luyện để quan sát và so sánh.
- Dự đoán dữ liệu tập test bằng cả 2 mô hình (mô hình thư viện thì dùng hàm predict() của thư viện, mô hình tự viết dùng hàm dự đoán tự viết), in ra kết quả bằng Dataframe như trong bài thực hành trên lớp.
- Tính RMSE trên tập test cho cả 2 mô hình và so sánh.

## Dữ liệu 

Tập dữ liệu giá nhà ở Boston đã có sẵn trên sklearn, dữ liệu đã được chuẩn hóa và chia thành tập train, tập test

In [117]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import math 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn import datasets, linear_model
from sklearn.datasets import fetch_openml
from sklearn.metrics import mean_squared_error, r2_score

# Đọc dữ liệu 

Dữ liệu về giá nhà ở Boston được hỗ trợ bởi sklearn, đọc dữ liệu thông qua hàm `datasets.load_boston()` 

Xem thêm các bộ dữ liệu khác tại https://scikit-learn.org/stable/datasets/index.html#toy-datasets. 

Dữ liệu được chia thành các thành phần data và target như tập diabetes. Dữ liệu cũng đã được chuẩn hóa, chỉ cần gọi ra và huấn luyện

In [118]:
# lay du lieu dataset - du lieu ve giá nhà
dataset = fetch_openml(name="house_prices", as_frame=True)
print("Số chiều dữ liệu input: ", dataset.data.shape)
print("Số chiều dữ liệu target: ", dataset.target.shape)
print()

print("5 mẫu dữ liệu đầu tiên:")
print("input: ", dataset.data[:5])
print("target: ",dataset.target[:5])

Số chiều dữ liệu input:  (1460, 80)
Số chiều dữ liệu target:  (1460,)

5 mẫu dữ liệu đầu tiên:
input:     Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... ScreenPorch PoolArea PoolQC Fence MiscFeature  \
0         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   
1         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   
2         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   
3         Lvl    AllPub  ...           0        0    NaN   NaN         NaN   
4         Lvl    AllPub  ...           0        0    NaN   NaN

**Chia dữ liệu làm 2 phần training 362 mẫu và testing 80 mẫu** 

In [119]:
# cat nho du lieu, lay 1 phan cho qua trinh thu nghiem,
# chia train test cac mau du lieu
# dataset_X = dataset.data[:, np.newaxis, 2]
dataset_X = dataset.data



In [120]:
dataset_X

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,8,2007,WD,Normal
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal


In [121]:
dataset_X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 80 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [122]:
for col in dataset_X.columns:
    print(dataset_X[col].value_counts())

Id
1       1
982     1
980     1
979     1
978     1
       ..
485     1
484     1
483     1
482     1
1460    1
Name: count, Length: 1460, dtype: int64
MSSubClass
20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: count, dtype: int64
MSZoning
RL           1151
RM            218
FV             65
RH             16
'C (all)'      10
Name: count, dtype: int64
LotFrontage
60.0     143
70.0      70
80.0      69
50.0      57
75.0      53
        ... 
137.0      1
141.0      1
38.0       1
140.0      1
46.0       1
Name: count, Length: 110, dtype: int64
LotArea
7200     25
9600     24
6000     17
9000     14
8400     14
         ..
14601     1
13682     1
4058      1
17104     1
9717      1
Name: count, Length: 1073, dtype: int64
Street
Pave    1454
Grvl       6
Name: count, dtype: int64
Alley
Grvl    50
Pave    41
Name: count, dtype: int64
LotShape
Reg    925
IR1    484
IR

In [123]:
dataset_X = dataset_X.drop(columns = ["Id", "Alley", "Fireplaces", "PoolQC", "Fence", "MiscFeature", "Street", "Utilities", "BsmtHalfBath", "GarageCond", "PavedDrive"])

In [124]:
encoder = LabelEncoder()

In [125]:
for col in dataset_X.columns:
    if dataset_X[col].dtype == object:
        dataset_X[col] = encoder.fit_transform(dataset_X[col])

In [126]:
dataset_X

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,3,65.0,8450,3,3,4,0,5,2,...,61,0,0,0,0,0,2,2008,8,4
1,20,3,80.0,9600,3,3,2,0,24,1,...,0,0,0,0,0,0,5,2007,8,4
2,60,3,68.0,11250,0,3,4,0,5,2,...,42,0,0,0,0,0,9,2008,8,4
3,70,3,60.0,9550,0,3,0,0,6,2,...,35,272,0,0,0,0,2,2006,8,0
4,60,3,84.0,14260,0,3,2,0,15,2,...,84,0,0,0,0,0,12,2008,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,3,3,4,0,8,2,...,40,0,0,0,0,0,8,2007,8,4
1456,20,3,85.0,13175,3,3,4,0,14,2,...,0,0,0,0,0,0,2,2010,8,4
1457,70,3,66.0,9042,3,3,4,0,6,2,...,60,0,0,0,0,2500,5,2010,8,4
1458,20,3,68.0,9717,3,3,4,0,12,2,...,0,112,0,0,0,0,4,2010,8,4


In [127]:
for col in dataset_X.columns:
    dataset_X[col] = dataset_X[col].fillna(dataset_X[col].mean())

In [128]:
dataset_X_train = dataset_X[:1168]
dataset_y_train = dataset.target[:1168]

dataset_X_test = dataset_X[1168:]
dataset_y_test = dataset.target[1168:]

print(dataset_X_train.shape)
print(dataset_X_test.shape)

(1168, 69)
(292, 69)


In [129]:
scaler.fit_transform(dataset_X_train)
scaler.fit_transform(dataset_X_test)

array([[ 0.30332039, -0.07982944,  2.00976078, ...,  0.14815939,
         0.27555828,  0.18006003],
       [ 0.0623148 , -0.07982944,  1.92873968, ..., -1.31836743,
         0.27555828,  0.18006003],
       [ 0.54432597, -0.07982944,  0.22729642, ...,  0.14815939,
         0.27555828,  0.18006003],
       ...,
       [ 0.30332039, -0.07982944, -0.17780912, ...,  1.61468621,
         0.27555828,  0.18006003],
       [-0.90170754, -0.07982944, -0.09678801, ...,  1.61468621,
         0.27555828,  0.18006003],
       [-0.90170754, -0.07982944,  0.18678587, ...,  0.14815939,
         0.27555828,  0.18006003]])

In [139]:
dataset_X_train

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,60,3,65.000000,8450,3,3,4,0,5,2,...,61,0,0,0,0,0,2,2008,8,4
1,20,3,80.000000,9600,3,3,2,0,24,1,...,0,0,0,0,0,0,5,2007,8,4
2,60,3,68.000000,11250,0,3,4,0,5,2,...,42,0,0,0,0,0,9,2008,8,4
3,70,3,60.000000,9550,0,3,0,0,6,2,...,35,272,0,0,0,0,2,2006,8,0
4,60,3,84.000000,14260,0,3,2,0,15,2,...,84,0,0,0,0,0,12,2008,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1163,90,3,60.000000,12900,3,3,4,0,19,1,...,0,0,0,0,0,0,1,2008,8,2
1164,80,3,70.049958,16157,0,3,2,0,24,1,...,180,0,0,0,0,0,6,2007,8,4
1165,20,3,79.000000,9541,0,3,4,0,16,2,...,114,0,0,0,0,0,9,2009,6,5
1166,20,3,64.000000,10475,0,3,0,0,5,2,...,33,0,0,0,0,0,2,2010,8,4


# Xây dựng mô hình

## Xây dựng mô hình bằng thư viện

In [130]:
model = linear_model.LinearRegression()

## Xây dựng mô hình Linear Regression tự viết 

In [131]:
def linear_regression(dataset_X_train,dataset_y_train):
    one = np.ones((dataset_X_train.shape[0],1))
    Xbar = np.concatenate((one, dataset_X_train), axis = 1)

    A = np.dot(Xbar.T, Xbar)
    b = np.dot(Xbar.T, dataset_y_train)
    w_lr = np.dot(np.linalg.pinv(A), b)
    coef = w_lr[1:]
    intercept = w_lr[0]
    return coef, intercept

## Hàm test mô hình tự viết

In [155]:
def predict(intercept, coef, dataset_X_test):
    y_pred_ = [intercept] + coef.dot(dataset_X_test[0].T)
    for i in range(1, len(dataset_X_test)):
        y_pred = intercept + coef.dot(dataset_X_test[i].T)
        y_pred_= np.append(y_pred_,y_pred)
    return y_pred_

# Huấn luyện mô hình

## Huấn luyện mô hình của thư viện

In [133]:
model.fit(dataset_X_train, dataset_y_train)
print(model.coef_)
print(model.intercept_)

[-8.07035396e+01  2.91568324e+02  6.56465947e+01  4.63988970e-01
 -2.00639260e+02  1.17874201e+03  5.09515502e+01  2.30348966e+03
  3.37244035e+02 -2.02410143e+03 -1.17441167e+04 -1.90241836e+03
 -2.46341350e+02  9.87945534e+03  5.01391037e+03  3.01193793e+02
  8.66113765e+01  1.33673879e+01  2.87596128e+03  2.32671927e+02
 -2.09499319e+02  4.54165116e+03  2.80242549e+01 -8.53814248e+03
  1.20798309e+03  2.38798662e+03 -6.12815049e+03  2.92201730e+03
 -1.67178072e+03  8.02735525e+01  1.31643979e+01  8.45721097e+02
  1.12247785e+01 -5.70401380e+00  1.86851625e+01  1.69960158e+03
 -9.32536374e+02 -2.10957683e+03 -4.90245124e+02  2.15399404e+01
  1.73598077e+01 -1.01139060e+01  2.87858421e+01  1.83818468e+03
  2.63950111e+03  2.14094004e+03 -7.39934044e+03 -1.63667385e+04
 -7.21268981e+03  5.37476219e+03  3.77242976e+03 -2.10561123e+03
  2.05812639e+03 -3.48514453e+01  2.95812827e+02  1.99111762e+03
  1.94497759e+01  1.17523182e+03  1.71762864e+01 -2.23604219e+01
  9.82541146e+00  6.65496

## Training mô hình bằng Linear regression tự viết

In [134]:
coef, intercept = linear_regression(dataset_X_train, dataset_y_train)
print(coef)
print(intercept)

[-8.07035425e+01  2.91568307e+02  6.56465962e+01  4.63988970e-01
 -2.00639237e+02  1.17874204e+03  5.09515343e+01  2.30348964e+03
  3.37244041e+02 -2.02410142e+03 -1.17441168e+04 -1.90241828e+03
 -2.46341298e+02  9.87945536e+03  5.01391036e+03  3.01193787e+02
  8.66113756e+01  1.33674040e+01  2.87596123e+03  2.32671929e+02
 -2.09499321e+02  4.54165125e+03  2.80242550e+01 -8.53814261e+03
  1.20798306e+03  2.38798668e+03 -6.12815047e+03  2.92201740e+03
 -1.67178068e+03  8.02735503e+01  1.31632693e+01  8.45721082e+02
  1.12236499e+01 -5.70514236e+00  1.86862911e+01  1.69960156e+03
 -9.32536389e+02 -2.10957684e+03 -4.90245073e+02  2.15403621e+01
  1.73602294e+01 -1.01134845e+01  2.87854203e+01  1.83818482e+03
  2.63950126e+03  2.14094013e+03 -7.39934044e+03 -1.63667385e+04
 -7.21268983e+03  5.37476216e+03  3.77242977e+03 -2.10561120e+03
  2.05812640e+03 -3.48514475e+01  2.95812793e+02  1.99111753e+03
  1.94497760e+01  1.17523191e+03  1.71762868e+01 -2.23604229e+01
  9.82541115e+00  6.65497

# Dự đoán các mẫu dữ liệu

## Dự đoán các mẫu dữ liệu theo mô hình của thư viện

In [135]:
dataset_y_pred_lib = model.predict(dataset_X_test)
pd.DataFrame(data = np.array([dataset_y_test, dataset_y_pred_lib,
                              abs(dataset_y_pred_lib - dataset_y_test)]).T,
                              columns = ["Thực tế", "Dự đoán", "Lệch"])

,Thực tế,Dự đoán,Lệch
0,235000.0,203896.534538,31103.465462
1,625000.0,454933.643886,170066.356114
2,171000.0,117553.423993,53446.576007
3,163000.0,167271.808983,4271.808983
4,171900.0,168324.284065,3575.715935
...,...,...,...
287,175000.0,171675.836789,3324.163211
288,210000.0,222533.992976,12533.992976
289,266500.0,247575.628661,18924.371339
290,142125.0,146225.496363,4100.496363


## Dự đoán các mẫu dữ liệu tính theo linear regression tự viết

In [147]:
dataset_X_test

,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,Condition1,...,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition
1168,70,3,120.0,13728,3,3,0,0,7,2,...,0,0,0,90,0,0,7,2008,8,4
1169,60,3,118.0,35760,0,3,1,0,15,2,...,76,0,0,0,0,0,7,2006,8,4
1170,80,3,76.0,9880,3,3,4,0,11,2,...,0,0,0,0,576,0,7,2008,8,4
1171,20,3,76.0,9120,3,3,4,0,12,2,...,0,0,0,288,0,1400,11,2008,8,4
1172,160,1,35.0,4017,0,3,4,0,21,2,...,54,0,0,0,0,0,3,2008,8,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,3,62.0,7917,3,3,4,0,8,2,...,40,0,0,0,0,0,8,2007,8,4
1456,20,3,85.0,13175,3,3,4,0,14,2,...,0,0,0,0,0,0,2,2010,8,4
1457,70,3,66.0,9042,3,3,4,0,6,2,...,60,0,0,0,0,2500,5,2010,8,4
1458,20,3,68.0,9717,3,3,4,0,12,2,...,0,112,0,0,0,0,4,2010,8,4


In [156]:
dataset_y_pred = predict(intercept, coef, np.array(dataset_X_test))
pd.DataFrame(data=np.array([dataset_y_test, dataset_y_pred,
                            abs(dataset_y_test - dataset_y_pred)]).T,
             columns=["Thực tế", "Dự đoán", "Lệch"]) 

,Thực tế,Dự đoán,Lệch
0,235000.0,203896.534861,31103.465139
1,625000.0,454933.645259,170066.354741
2,171000.0,117553.422518,53446.577482
3,163000.0,167271.809089,4271.809089
4,171900.0,168324.284359,3575.715641
...,...,...,...
287,175000.0,171675.837138,3324.162862
288,210000.0,222533.992075,12533.992075
289,266500.0,247575.627843,18924.372157
290,142125.0,146225.495453,4100.495453


## Đánh giá mô hình linear regression của thư viện

In [157]:
loss = math.sqrt(mean_squared_error(dataset_y_test, dataset_y_pred_lib))
loss

47388.07068142908

## Đánh giá mô hình linear regression tự viết

In [158]:
loss = math.sqrt(mean_squared_error(dataset_y_test, dataset_y_pred))
loss

47388.07069534216